In [1]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.layers import LSTM, GRU, SimpleRNN
from keras import backend as K


Using TensorFlow backend.


In [2]:
traindata = pd.read_csv('C:/Users/Aditya Kyatham/Documents/BE FINAL/nsl-kdd/NSL_KDD-master/Small Training Set_5class_train.csv', index_col=None)
testdata = pd.read_csv('C:/Users/Aditya Kyatham/Documents/BE FINAL/nsl-kdd/NSL_KDD-master/Small Training Set_5class_test.csv', index_col=None)

traindata.head()

,duration,"""protocol_type""","""service""",flag,"""src_bytes""","""dst_bytes""",land,wrong_fragment,urgent,"""hot""",...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_diff_srv_rate.1,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,S1,278,7958,0,0,0,0,...,255,1.00,0.00,0.33,0.03,0.33,0.0,0.0,0.0,normal
1,0,tcp,http,SF,241,851,0,0,0,0,...,255,1.00,0.00,0.01,0.03,0.00,0.0,0.0,0.0,normal
2,0,udp,private,SF,105,146,0,0,0,0,...,239,0.94,0.01,0.01,0.00,0.00,0.0,0.0,0.0,normal
3,0,tcp,http,SF,367,5199,0,0,0,0,...,255,1.00,0.00,0.01,0.05,0.00,0.0,0.0,0.0,normal
4,21263,tcp,private,RSTR,1,0,0,0,0,0,...,2,0.01,0.66,1.00,0.00,0.00,0.0,1.0,1.0,probe


In [3]:
from sklearn.preprocessing import LabelEncoder
encodings = dict()
for c in traindata.columns:
    #print df[c].dtype
    if traindata[c].dtype == "object":
        encodings[c] = LabelEncoder() #to give numerical label to char type labels.
        encodings[c]
        traindata[c] = encodings[c].fit_transform(traindata[c])

In [4]:
encodings1 = dict()
for c in testdata.columns:
    #print df[c].dtype
    if testdata[c].dtype == "object":
        encodings1[c] = LabelEncoder() #to give numerical label to char type labels.
        encodings1[c]
        testdata[c] = encodings1[c].fit_transform(testdata[c])

In [5]:
X = traindata.iloc[:,0:41]
Y = traindata.iloc[:,41]
C = testdata.iloc[:,41]
T = testdata.iloc[:,0:41]
Y.head()

0    1
1    1
2    1
3    1
4    2
Name: label, dtype: int32

In [6]:
X=X.drop(['land','su_attempted','num_outbound_cmds','is_host_login','urgent','num_failed_logins','su_attempted','num_file_creations','num_shells','srv_diff_host_rate'], axis = 1) 
T=T.drop(['land','su_attempted','num_outbound_cmds','is_host_login','urgent','num_failed_logins','su_attempted','num_file_creations','num_shells','srv_diff_host_rate'], axis = 1) 

In [7]:
T.head()

,duration,"""protocol_type""","""service""",flag,"""src_bytes""","""dst_bytes""",wrong_fragment,"""hot""",logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_diff_srv_rate.1,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,3047,2,26,5,146,105,0,0,0,0,...,255,1,0.00,0.69,0.96,0.00,0.0,0.0,0.0,0.0
1,0,1,15,5,202,1923,0,0,1,0,...,21,255,1.00,0.00,0.05,0.02,0.0,0.0,0.0,0.0
2,0,2,6,5,44,72,0,0,0,0,...,255,254,1.00,0.01,0.00,0.00,0.0,0.0,0.0,0.0
3,0,1,27,3,0,0,0,0,0,0,...,255,14,0.05,0.08,0.00,0.00,1.0,1.0,0.0,0.0
4,0,1,27,2,0,0,0,0,0,0,...,255,1,0.00,0.82,1.00,0.00,0.0,0.0,1.0,1.0


In [8]:
scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
# summarize transformed data
np.set_printoptions(precision=3)
print(trainX[0:5,:])

[[0.000e+00 1.255e-04 2.636e-03 5.021e-04 3.489e-02 9.989e-01 0.000e+00
  0.000e+00 1.255e-04 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  1.255e-04 2.510e-04 1.255e-04 6.276e-05 0.000e+00 0.000e+00 1.255e-04
  0.000e+00 3.766e-04 3.201e-02 1.255e-04 0.000e+00 4.142e-05 3.766e-06
  4.142e-05 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 1.075e-03 2.257e-02 5.375e-03 2.591e-01 9.148e-01 0.000e+00
  0.000e+00 1.075e-03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  1.827e-02 3.762e-02 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.075e-03
  0.000e+00 1.365e-01 2.741e-01 1.075e-03 0.000e+00 1.075e-05 3.225e-05
  0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 5.062e-03 1.012e-01 1.265e-02 2.657e-01 3.695e-01 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  7.592e-03 5.062e-03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.696e-03
  1.696e-03 6.454e-01 6.049e-01 2.379e-03 2.531e-05 2.531e-05 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000

In [9]:
scaler = Normalizer().fit(T)
testT = scaler.transform(T)
# summarize transformed data
np.set_printoptions(precision=3)
#print(testT[0:5,:])

In [10]:
y_train1 = np.array(Y)
y_test1 = np.array(C)

y_train= to_categorical(y_train1)
y_test= to_categorical(y_test1)

In [11]:

# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
X_test = np.reshape(testT, (testT.shape[0],testT.shape[1],1))
batch_size = 64

In [12]:
X_train.shape

(808, 32, 1)

In [17]:

cnn = Sequential()
cnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(32, 1)))
cnn.add(Convolution1D(64, 3, border_mode="same", activation="relu"))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(Flatten())
cnn.add(Dense(64, activation="relu"))
cnn.add(Dropout(0.5))
cnn.add(Dense(5, activation="softmax"))



C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(32, 1), padding="same")`
  
C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  after removing the cwd from sys.path.


In [29]:
lstm_output_size = 70

cnn1 = Sequential()
cnn1.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(32, 1)))
cnn1.add(Convolution1D(64, 3, border_mode="same", activation="relu"))
cnn1.add(MaxPooling1D(pool_length=(2)))
cnn1.add(Convolution1D(128, 3, border_mode="same", activation="relu"))
cnn1.add(Convolution1D(128, 3, border_mode="same", activation="relu"))
cnn1.add(MaxPooling1D(pool_length=(2)))
cnn1.add(Convolution1D(256, 3, border_mode="same", activation="relu"))
cnn1.add(Convolution1D(256, 3, border_mode="same", activation="relu"))
cnn1.add(MaxPooling1D(pool_length=(2)))
cnn1.add(LSTM(lstm_output_size))
cnn1.add(Dropout(0.1))
cnn1.add(Dense(5, activation="softmax"))


C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(32, 1), padding="same")`
  after removing the cwd from sys.path.
C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
  """
C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  
C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, activation="relu", padding="same")`
  import sys
C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\si

In [18]:
# try using different optimizers and different optimizer configs
import time
start = time.time()
import tensorflow as tf
cnn.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
#checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/lstm1layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
#csv_logger = CSVLogger('training_set_iranalysis.csv',separator=',', append=False)
cnn.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1000, validation_data=(X_test, y_test))#,callbacks=[checkpointer,csv_logger])
#model.save("kddresults/lstm1layer/fullmodel/lstm1layer_model.hdf5")

#loss, accuracy = model.evaluate(X_test, y_test)
#print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
#y_pred = model.predict_classes(X_test)
#np.savetxt('kddresults/lstm1layer/lstm1predicted.txt', np.transpose([y_test1,y_pred]), fmt='%01d')
end = time.time()
print('exec time = '+str(end-start))


C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:8: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 808 samples, validate on 203 samples
Epoch 1/1000
808/808 [==============================] - 1s 1ms/step - loss: 1.3756 - accuracy: 0.7116 - precision_2: 0.1538 - recall_2: 5.7617e-04 - val_loss: 1.0060 - val_accuracy: 0.7980 - val_precision_2: 1.0000 - val_recall_2: 0.0121
Epoch 2/1000
808/808 [==============================] - 0s 335us/step - loss: 0.7957 - accuracy: 0.7797 - precision_2: 0.8603 - recall_2: 0.1632 - val_loss: 0.7167 - val_accuracy: 0.7635 - val_precision_2: 0.8209 - val_recall_2: 0.3139
Epoch 3/1000
808/808 [==============================] - 0s 367us/step - loss: 0.6329 - accuracy: 0.8082 - precision_2: 0.8187 - recall_2: 0.4036 - val_loss: 0.5654 - val_accuracy: 0.7931 - val_precision_2: 0.8266 - val_recall_2: 0.4729
Epoch 4/1000
808/808 [==============================] - 0s 375us/step - loss: 0.5522 - accuracy: 0.8379 - precision_2: 0.8305 - recall_2: 0.5155 - val_loss: 0.5323 - val_accuracy: 0.7931 - val_precision_2: 0.8367 - val_recall_2: 0.5527
Epoch 5/

808/808 [==============================] - 0s 312us/step - loss: 0.2525 - accuracy: 0.9121 - precision_2: 0.9027 - recall_2: 0.8364 - val_loss: 0.3125 - val_accuracy: 0.9113 - val_precision_2: 0.9030 - val_recall_2: 0.8374
Epoch 36/1000
808/808 [==============================] - 0s 296us/step - loss: 0.2462 - accuracy: 0.9158 - precision_2: 0.9032 - recall_2: 0.8382 - val_loss: 0.3273 - val_accuracy: 0.9064 - val_precision_2: 0.9035 - val_recall_2: 0.8391
Epoch 37/1000
808/808 [==============================] - 0s 416us/step - loss: 0.2453 - accuracy: 0.9220 - precision_2: 0.9039 - recall_2: 0.8401 - val_loss: 0.3229 - val_accuracy: 0.9064 - val_precision_2: 0.9041 - val_recall_2: 0.8411
Epoch 38/1000
808/808 [==============================] - 0s 307us/step - loss: 0.2414 - accuracy: 0.9183 - precision_2: 0.9043 - recall_2: 0.8420 - val_loss: 0.3021 - val_accuracy: 0.9064 - val_precision_2: 0.9047 - val_recall_2: 0.8429
Epoch 39/1000
808/808 [==============================] - 0s 303us/

808/808 [==============================] - 0s 310us/step - loss: 0.1769 - accuracy: 0.9369 - precision_2: 0.9179 - recall_2: 0.8756 - val_loss: 0.3158 - val_accuracy: 0.9163 - val_precision_2: 0.9181 - val_recall_2: 0.8761
Epoch 70/1000
808/808 [==============================] - 0s 303us/step - loss: 0.2029 - accuracy: 0.9344 - precision_2: 0.9183 - recall_2: 0.8764 - val_loss: 0.2578 - val_accuracy: 0.9212 - val_precision_2: 0.9184 - val_recall_2: 0.8767
Epoch 71/1000
808/808 [==============================] - 0s 338us/step - loss: 0.1871 - accuracy: 0.9319 - precision_2: 0.9185 - recall_2: 0.8770 - val_loss: 0.2690 - val_accuracy: 0.9163 - val_precision_2: 0.9187 - val_recall_2: 0.8774
Epoch 72/1000
808/808 [==============================] - 0s 338us/step - loss: 0.1769 - accuracy: 0.9381 - precision_2: 0.9188 - recall_2: 0.8777 - val_loss: 0.2865 - val_accuracy: 0.9163 - val_precision_2: 0.9189 - val_recall_2: 0.8781
Epoch 73/1000
808/808 [==============================] - 0s 304us/

Epoch 104/1000
808/808 [==============================] - 0s 245us/step - loss: 0.1456 - accuracy: 0.9493 - precision_2: 0.9266 - recall_2: 0.8946 - val_loss: 0.2806 - val_accuracy: 0.9261 - val_precision_2: 0.9267 - val_recall_2: 0.8948
Epoch 105/1000
808/808 [==============================] - 0s 232us/step - loss: 0.1339 - accuracy: 0.9542 - precision_2: 0.9269 - recall_2: 0.8951 - val_loss: 0.2604 - val_accuracy: 0.9310 - val_precision_2: 0.9270 - val_recall_2: 0.8953
Epoch 106/1000
808/808 [==============================] - 0s 250us/step - loss: 0.1447 - accuracy: 0.9505 - precision_2: 0.9272 - recall_2: 0.8955 - val_loss: 0.2782 - val_accuracy: 0.9261 - val_precision_2: 0.9273 - val_recall_2: 0.8957
Epoch 107/1000
808/808 [==============================] - 0s 246us/step - loss: 0.1397 - accuracy: 0.9480 - precision_2: 0.9273 - recall_2: 0.8959 - val_loss: 0.2673 - val_accuracy: 0.9310 - val_precision_2: 0.9275 - val_recall_2: 0.8962
Epoch 108/1000
808/808 [========================

...

808/808 [==============================] - 0s 404us/step - loss: 0.0151 - accuracy: 0.9938 - precision_2: 0.9729 - recall_2: 0.9659 - val_loss: 0.1288 - val_accuracy: 0.9852 - val_precision_2: 0.9729 - val_recall_2: 0.9659
Epoch 853/1000
808/808 [==============================] - 0s 378us/step - loss: 0.0140 - accuracy: 0.9950 - precision_2: 0.9729 - recall_2: 0.9659 - val_loss: 0.1271 - val_accuracy: 0.9852 - val_precision_2: 0.9729 - val_recall_2: 0.9659
Epoch 854/1000
808/808 [==============================] - 0s 427us/step - loss: 0.0110 - accuracy: 0.9963 - precision_2: 0.9729 - recall_2: 0.9659 - val_loss: 0.1298 - val_accuracy: 0.9803 - val_precision_2: 0.9729 - val_recall_2: 0.9660
Epoch 855/1000
808/808 [==============================] - 0s 453us/step - loss: 0.0265 - accuracy: 0.9901 - precision_2: 0.9729 - recall_2: 0.9660 - val_loss: 0.1756 - val_accuracy: 0.9852 - val_precision_2: 0.9730 - val_recall_2: 0.9660
Epoch 856/1000
808/808 [==============================] - 0s 44

808/808 [==============================] - 0s 344us/step - loss: 0.0134 - accuracy: 0.9963 - precision_2: 0.9736 - recall_2: 0.9669 - val_loss: 0.1206 - val_accuracy: 0.9852 - val_precision_2: 0.9736 - val_recall_2: 0.9669
Epoch 887/1000
808/808 [==============================] - 0s 372us/step - loss: 0.0210 - accuracy: 0.9864 - precision_2: 0.9736 - recall_2: 0.9669 - val_loss: 0.1382 - val_accuracy: 0.9852 - val_precision_2: 0.9736 - val_recall_2: 0.9669
Epoch 888/1000
808/808 [==============================] - 0s 411us/step - loss: 0.0244 - accuracy: 0.9913 - precision_2: 0.9737 - recall_2: 0.9669 - val_loss: 0.1304 - val_accuracy: 0.9852 - val_precision_2: 0.9737 - val_recall_2: 0.9669
Epoch 889/1000
808/808 [==============================] - 0s 370us/step - loss: 0.0130 - accuracy: 0.9938 - precision_2: 0.9737 - recall_2: 0.9669 - val_loss: 0.1240 - val_accuracy: 0.9852 - val_precision_2: 0.9737 - val_recall_2: 0.9669
Epoch 890/1000
808/808 [==============================] - 0s 41

808/808 [==============================] - 0s 406us/step - loss: 0.0148 - accuracy: 0.9963 - precision_2: 0.9743 - recall_2: 0.9677 - val_loss: 0.1508 - val_accuracy: 0.9803 - val_precision_2: 0.9743 - val_recall_2: 0.9678
Epoch 921/1000
808/808 [==============================] - 0s 442us/step - loss: 0.0155 - accuracy: 0.9950 - precision_2: 0.9743 - recall_2: 0.9678 - val_loss: 0.1422 - val_accuracy: 0.9852 - val_precision_2: 0.9743 - val_recall_2: 0.9678
Epoch 922/1000
808/808 [==============================] - 0s 435us/step - loss: 0.0120 - accuracy: 0.9963 - precision_2: 0.9743 - recall_2: 0.9678 - val_loss: 0.1625 - val_accuracy: 0.9803 - val_precision_2: 0.9744 - val_recall_2: 0.9678
Epoch 923/1000
808/808 [==============================] - 0s 437us/step - loss: 0.0187 - accuracy: 0.9889 - precision_2: 0.9744 - recall_2: 0.9678 - val_loss: 0.1735 - val_accuracy: 0.9803 - val_precision_2: 0.9744 - val_recall_2: 0.9678
Epoch 924/1000
808/808 [==============================] - 0s 40

808/808 [==============================] - 0s 353us/step - loss: 0.0113 - accuracy: 0.9963 - precision_2: 0.9749 - recall_2: 0.9686 - val_loss: 0.1680 - val_accuracy: 0.9852 - val_precision_2: 0.9749 - val_recall_2: 0.9686
Epoch 955/1000
808/808 [==============================] - 0s 388us/step - loss: 0.0121 - accuracy: 0.9938 - precision_2: 0.9749 - recall_2: 0.9686 - val_loss: 0.1594 - val_accuracy: 0.9852 - val_precision_2: 0.9750 - val_recall_2: 0.9686
Epoch 956/1000
808/808 [==============================] - 0s 338us/step - loss: 0.0117 - accuracy: 0.9975 - precision_2: 0.9750 - recall_2: 0.9686 - val_loss: 0.1651 - val_accuracy: 0.9754 - val_precision_2: 0.9750 - val_recall_2: 0.9686
Epoch 957/1000
808/808 [==============================] - 0s 335us/step - loss: 0.0148 - accuracy: 0.9926 - precision_2: 0.9750 - recall_2: 0.9686 - val_loss: 0.1590 - val_accuracy: 0.9803 - val_precision_2: 0.9750 - val_recall_2: 0.9687
Epoch 958/1000
808/808 [==============================] - 0s 34

808/808 [==============================] - 0s 474us/step - loss: 0.0138 - accuracy: 0.9950 - precision_2: 0.9755 - recall_2: 0.9693 - val_loss: 0.1432 - val_accuracy: 0.9754 - val_precision_2: 0.9755 - val_recall_2: 0.9694
Epoch 989/1000
808/808 [==============================] - 0s 444us/step - loss: 0.0081 - accuracy: 0.9988 - precision_2: 0.9755 - recall_2: 0.9694 - val_loss: 0.1452 - val_accuracy: 0.9754 - val_precision_2: 0.9756 - val_recall_2: 0.9694
Epoch 990/1000
808/808 [==============================] - 0s 439us/step - loss: 0.0180 - accuracy: 0.9926 - precision_2: 0.9756 - recall_2: 0.9694 - val_loss: 0.1740 - val_accuracy: 0.9655 - val_precision_2: 0.9756 - val_recall_2: 0.9694
Epoch 991/1000
808/808 [==============================] - 0s 601us/step - loss: 0.0154 - accuracy: 0.9926 - precision_2: 0.9756 - recall_2: 0.9694 - val_loss: 0.2013 - val_accuracy: 0.9754 - val_precision_2: 0.9756 - val_recall_2: 0.9694
Epoch 992/1000
808/808 [==============================] - 0s 45